In [12]:
# Libraries
from huggingface_hub import HfApi

In [13]:
api = HfApi()

# Search for notebooks related to software engineering

## Search by 'search' function
### Software Engineering

In [14]:
# Search for models related to software engineering
software_engineering_models = api.list_models(search="software engineering",cardData=True, sort='downloads', direction=-1)

# Example for analyzing model cards: https://github.com/Weixin-Liang/AI-model-card-analysis-HuggingFace/blob/main/api_example.ipynb

for model in software_engineering_models:
    model_info = api.model_info(model.id)
    print(f"Model name: {model_info.id}")
    print(f"Downloads: {model_info.downloads}")
    print(f"Downloads all time: {model_info.downloads_all_time}")
    print(f"Model tags: {model_info.tags}", "\n")

Model name: justinlamlamlam/softwareengineering
Downloads: 6
Downloads all time: None
Model tags: ['transformers', 'safetensors', 'bert', 'text-classification', 'arxiv:1910.09700', 'autotrain_compatible', 'endpoints_compatible', 'region:us'] 

Model name: burakkececi/bert-turkish-software-engineering
Downloads: 4
Downloads all time: None
Model tags: ['transformers', 'safetensors', 'bert', 'fill-mask', 'tr', 'license:mit', 'autotrain_compatible', 'endpoints_compatible', 'region:us'] 

Model name: burakkececi/bert-software-engineering
Downloads: 2
Downloads all time: None
Model tags: ['transformers', 'safetensors', 'bert', 'fill-mask', 'en', 'license:mit', 'autotrain_compatible', 'endpoints_compatible', 'region:us'] 



### Software Development

In [15]:
software_development_models = api.list_models(search="software development",cardData=True, sort='downloads', direction=-1)

for model in software_development_models:
    model_info = api.model_info(model.id)
    print(f"Model name: {model_info.id}")
    print(f"Downloads: {model_info.downloads}")
    print(f"Downloads all time: {model_info.downloads_all_time}")
    print(f"Model tags: {model_info.tags}","\n")

Model name: Jayian/software-development
Downloads: 0
Downloads all time: None
Model tags: ['license:creativeml-openrail-m', 'region:us'] 



## Search by tags
### Software Engineering

In [16]:
software_engineering_models_tags = api.list_models(tags=["software engineering"], cardData=True, sort='downloads', direction=-1)


for model in software_engineering_models_tags:
    model_info = api.model_info(model.id)
    print(f"Model name: {model_info.id}")
    print(f"Downloads: {model_info.downloads}")
    print(f"Downloads all time: {model_info.downloads_all_time}")
    print(f"Model tags: {model_info.tags}", "\n")

Model name: taidng/wikiser-bert-large
Downloads: 30
Downloads all time: None
Model tags: ['transformers', 'pytorch', 'safetensors', 'bert', 'token-classification', 'software engineering', 'ner', 'named-entity recognition', 'en', 'dataset:wikiser', 'arxiv:2308.10564', 'license:apache-2.0', 'autotrain_compatible', 'endpoints_compatible', 'region:us'] 

Model name: taidng/wikiser-bert-base
Downloads: 6
Downloads all time: None
Model tags: ['transformers', 'pytorch', 'safetensors', 'bert', 'token-classification', 'software engineering', 'ner', 'named-entity recognition', 'en', 'dataset:wikiser', 'arxiv:2308.10564', 'license:apache-2.0', 'autotrain_compatible', 'endpoints_compatible', 'region:us'] 

Model name: thearod5/nl-bert
Downloads: 3
Downloads all time: None
Model tags: ['transformers', 'safetensors', 'bert', 'text-classification', 'software engineering', 'software traceability', 'arxiv:2207.01084', 'autotrain_compatible', 'endpoints_compatible', 'region:us'] 

